# Project 2: Web Traffic Analysis
**This is the second of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-11-10, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
In this project your task is to analyze a stream of log entries. A log entry consists of an [IP address](https://en.wikipedia.org/wiki/IP_address) and a [domain name](https://en.wikipedia.org/wiki/Domain_name). For example, a log line may look as follows:

`192.168.0.1 somedomain.dk`

One log line is the result of the event that the domain name was visited by someone having the corresponding IP address. Your task is to analyze the traffic on a number of domains. Counting the number of unique IPs seen on a domain doesn't correspond to the exact number of unique visitors, but it is a good estimate.

Specifically, you should answer the following questions from the stream of log entries.

- How many unique IPs are there in the stream?
- How many unique IPs are there for each domain?
- How many times was IP X seen on domain Y? (for some X and Y provided at run time)

**The answers to these questions can be approximate!**

You should also try to answer one or more of the following, more advanced, questions. The answers to these should also be approximate.

- How many unique IPs are there for the domains $d_1, d_2, \ldots$?
- How many times was IP X seen on domains $d_1, d_2, \ldots$?
- What are the X most frequent IPs in the stream?

You should use algorithms and data structures that you've learned about in the lectures, and you should provide your own implementations of these.

Furthermore, you are expected to:

- Document the accuracy of your answers when using algorithms that give approximate answers
- Argue why you are using certain parameters for your data structures

This notebook is in three parts. In the first part you are given an example of how to read from the stream (which for the purpose of this project is a remote file). In the second part you should implement the algorithms and data structures that you intend to use, and in the last part you should use these for analyzing the stream.

## Reading the stream
The following code reads a remote file line by line. It is wrapped in a generator to make it easier to extend. You may modify this if you want to, but your solution should remain parametrized, so that your notebook can be run without having to consume the entire file.

In [138]:
import urllib

def stream(n):
    i = 0
    with urllib.request.urlopen('https://files.dtu.dk/fss/public/link/public/stream/read/traffic_2?linkToken=_DcyO-U3MjjuNzI-&itemName=traffic_2') as f:
        for line in f:
            element = line.rstrip().decode("utf-8")
            yield element
            i += 1
            if i == n:
                break

In [139]:
STREAM_SIZE = 1000
web_traffic_stream = stream(STREAM_SIZE)

## Data structures

In [187]:
import mmh3
import random
import sys
import math
import statistics as s
# number of hash functions (63 bit (signed))

def hashElement(element, seed):
    a,b = mmh3.hash64(element, seed, signed=False)
    return a



def buildDataStructure(stream, streamSize, rnd):
    m = 63;
    M = [0]*m
    seed = 42
    if(rnd):
        seed = random.randint(0,sys.maxsize/2)
    #Hash every element in the stream
    for e in stream:
        hE = hashElement(e, seed)
        bitstring = hE
        size_upper_part = math.floor(math.log(bitstring.bit_length(),2))
        size_lower_part = bitstring.bit_length() - size_upper_part
        upperpart = bitstring >> size_lower_part
        lowerpart = bitstring & ((2 ** size_lower_part) -1)
        # p is the position of the left most bit
        p = 64 - int(math.log(lowerpart,2))
        # j is the integer representation of the upperpart +1
        j = upperpart - 1
        #if(p>M[j]): print(str(p)); print(bin(upperpart))
        #if(j>=63):
            #print("upper: " + str(upperpart))
            #print("lower: " + bin(lowerpart))
            #print("hash:  " + bin(hE))
        M[j] = max(M[j],p)
    return M

def getHarmonicMean(dataStucture):
    return s.harmonic_mean(dataStucture)


web_traffic_stream = stream(STREAM_SIZE)
ds = buildDataStructure(web_traffic_stream, STREAM_SIZE, True)
print(ds)
getHarmonicMean(ds)

# It seams to work now. Except the getHarmonicMean. The data structure returns 0 in the first 15 slots. 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 11, 12, 14, 12, 14, 22, 13, 16, 13, 17, 20, 19, 16, 13, 14, 10, 15, 17, 9, 11, 13, 8, 11, 12, 11, 10, 11, 13, 13, 11, 10, 13, 12, 9, 11, 15, 9, 14, 11, 12, 12, 11, 10, 10, 11, 10, 13]


0

In [72]:
counter=0
o = 100000

for i in range(o):
    hE = hashElement("hello", random.randint(0,sys.maxsize/2))
    if(hE <0): counter +=1
        
print(counter)
print(counter/o*100)

0
0.0


## Analysis

In [120]:
hE = hashElement("hello", random.randint(0,sys.maxsize/2))
#print("hash: "+str(hE))

bitstring = hE
size_upper_part = math.floor(math.log(bitstring.bit_length(),2))
size_lower_part = bitstring.bit_length() - size_upper_part
upperpart = bitstring >> size_lower_part
lowerpart = bitstring & ((2 ** size_lower_part) -1)
print("org:    " + bin(bitstring))
print("upper:  " + bin(upperpart))
print("lower:  " + bin(lowerpart))
print("Position upper: " + str(upperpart.bit_length()))
print("Position lower: " + str(lowerpart.bit_length()))

org:    0b11111110011011101111001010101
upper:  0b1111
lower:  0b1110011011101111001010101
Position upper: 4
Position lower: 25


In [5]:
import random 
import sys

print(random.randint(-sys.maxsize-1,sys.maxsize));

-3499686339677874721
